In [1]:
import numpy as np
import pandas as pd
import random
from torch.utils import data
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms,datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils import data
import sys,os
sys.path.append(r"/./models")
from GAFM import train_GAFM
from Marvell import train_marvell
from Vanilla import train_vanilla
from MaxNorm import train_maxnorm
from bases import process_data,DataSet,seed_everything,get_logger
from tqdm import trange
import datetime
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

/home/yujin/.conda/envs/yjenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0
cuda:0
cuda:0
cuda:0


In [2]:
def get_data(random_state,dataname="ISIC"):
    config = {
      "batch_size": 128}
    
    train_transform = transforms.Compose([transforms.Resize((64, 64)),
                                          # transforms.Grayscale(num_output_channels=1),
                                          transforms.Grayscale(num_output_channels=1),
                                          transforms.ToTensor()
                                          ])
    train_data = datasets.ImageFolder('/home/yujin/GAN/data/train', train_transform)
    test_data = datasets.ImageFolder('/home/yujin/GAN/data/test', train_transform)
    train_loader = data.DataLoader(train_data, batch_size=config["batch_size"], shuffle=True, pin_memory=True)
    test_loader = data.DataLoader(test_data, batch_size=config["batch_size"], shuffle=True)
    features=1

    return train_loader, test_loader, features

In [3]:
##Parameters
dataname='ISIC'
log_path='/./logs'
# now = datetime.datetime.now().strftime('%b%d_%H-%M')+dataname+str(repeats)+str(lr)+str(Epochs)+'M'
now = datetime.datetime.now().strftime('%b%d_%H-%M')
log_dir = f'{log_path}/{now}'
isExist = os.path.exists(log_dir)
if not isExist:
  os.mkdir(log_dir)
logger= open(os.path.join(log_dir, 'log.txt'), 'w')

In [4]:
Epochs=250
repeats=1
lr=1e-6
print('Training MaxNorm...')
train_auc_list_maxnorm,test_auc_list_maxnorm,train_tvd_list_maxnorm,na_leak_auc_list_maxnorm,ma_leak_auc_list_maxnorm,cos_leak_auc_list_maxnorm=[],[],[],[],[],[]
for i in trange(repeats):
  seed_everything(seed=i)
  train_loader,test_loader,features=get_data(i)
  train_auc_maxnorm,test_auc_maxnorm,train_tvd_maxnorm,na_leak_auc_maxnorm,ma_leak_auc_maxnorm,cos_leak_auc_maxnorm,splitnn_maxnorm=train_maxnorm(Epochs=Epochs,features=features,train_loader=train_loader,test_loader=test_loader,lr=lr,info=True,logger=logger)
  train_auc_list_maxnorm.append(train_auc_maxnorm)
  test_auc_list_maxnorm.append(test_auc_maxnorm)
  train_tvd_list_maxnorm.append(train_tvd_maxnorm)
  na_leak_auc_list_maxnorm.append(na_leak_auc_maxnorm)
  ma_leak_auc_list_maxnorm.append(ma_leak_auc_maxnorm)
  cos_leak_auc_list_maxnorm.append(cos_leak_auc_maxnorm)
  if na_leak_auc_maxnorm<best:
    best=na_leak_auc_maxnorm
    maxnorm_model=splitnn_maxnorm

print('MaxNorm Training AUC',(train_auc_list_maxnorm))
print('MaxNorm Testing AUC',(test_auc_list_maxnorm))
print('MaxNorm TVD',(train_tvd_list_maxnorm))
print('MaxNorm NA Leak AUC',(na_leak_auc_list_maxnorm))
print('MaxNorm MA Leak AUC',(ma_leak_auc_list_maxnorm))
print('MaxNorm Median Leak AUC',(cos_leak_auc_list_maxnorm))

print('MaxNorm Mean Training AUC',np.mean(train_auc_list_maxnorm),np.std(train_auc_list_maxnorm))
print('MaxNorm Mean Testing AUC',np.mean(test_auc_list_maxnorm),np.std(test_auc_list_maxnorm))
print('MaxNorm Mean TVD',np.mean(train_tvd_list_maxnorm),np.std(train_tvd_list_maxnorm))
print('MaxNorm Mean NA Leak AUC',np.mean(na_leak_auc_list_maxnorm),np.std(na_leak_auc_list_maxnorm))
print('MaxNorm Mean MA Leak AUC',np.mean(ma_leak_auc_list_maxnorm),np.std(ma_leak_auc_list_maxnorm))
print('MaxNorm Mean Median Leak AUC',np.mean(cos_leak_auc_list_maxnorm),np.std(cos_leak_auc_list_maxnorm))


Training MaxNorm...


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0 Training Loss 0.005488883313603241 Training AUC 0.47025666016894085 Testing AUC 0.4797005599172805 NA Leak AUC 0.9709870741668988 MA Leak AUC 1.0 Median Leak AUC 1.0
Epoch 10 Training Loss 0.0054888043150830915 Training AUC 0.5738437529007705 Testing AUC 0.612745328016264 NA Leak AUC 0.9716861598440546 MA Leak AUC 1.0 Median Leak AUC 1.0
Epoch 20 Training Loss 0.005486623733420595 Training AUC 0.6268947832544323 Testing AUC 0.6486994910142749 NA Leak AUC 0.9667681936322288 MA Leak AUC 1.0 Median Leak AUC 1.0
Epoch 30 Training Loss 0.005487033924620448 Training AUC 0.6076644156688016 Testing AUC 0.6086620484181349 NA Leak AUC 0.9688828552863641 MA Leak AUC 1.0 Median Leak AUC 1.0
Epoch 40 Training Loss 0.005486776286014446 Training AUC 0.6083814861227141 Testing AUC 0.6139150863077244 NA Leak AUC 0.9695854218880535 MA Leak AUC 1.0 Median Leak AUC 1.0
Epoch 50 Training Loss 0.005486052516349448 Training AUC 0.616060405643739 Testing AUC 0.6137447916550554 NA Leak AUC 0.9695854218

100%|██████████| 1/1 [25:19<00:00, 1519.17s/it]

Epoch 249 Training Loss 0.005065771694735196 Training AUC 0.7128063213589528 Testing AUC 0.7216308012915114 NA Leak AUC 0.732184048083171 MA Leak AUC 0.9327746449456975 Median Leak AUC 0.9290204678362574
MaxNorm Training AUC [0.7128063213589528]
MaxNorm Testing AUC [0.7216308012915114]
MaxNorm TVD [0.05729943722941201]
MaxNorm NA Leak AUC [0.732184048083171]
MaxNorm MA Leak AUC [0.9327746449456975]
MaxNorm Median Leak AUC [0.9290204678362574]
MaxNorm Mean Training AUC 0.7128063213589528 0.0
MaxNorm Mean Testing AUC 0.7216308012915114 0.0
MaxNorm Mean TVD 0.05729943722941201 0.0
MaxNorm Mean NA Leak AUC 0.732184048083171 0.0
MaxNorm Mean MA Leak AUC 0.9327746449456975 0.0
MaxNorm Mean Median Leak AUC 0.9290204678362574 0.0
